In [1]:
from segnlp import Pipeline
from segnlp.datasets.am import PE
from segnlp.pretrained_features import GloveEmbeddings

Using backend: pytorch


In [2]:

# setting up the pipeline      
exp = Pipeline(
                id="lstm_er_pe", # will create folder at with id as name at ~/.segnlp/<id>/ 
                dataset=PE(
                    tasks=["seg+label", "link", "link_label"],
                    prediction_level="token",
                    sample_level="document",
                ),
                metric = "overlap_metric", # settting metric, can be found in segnlp.metrics
                model = "LSTM_ER",
                pretrained_features = [
                                GloveEmbeddings(),
                            ],
                #overwrite = True, # will remove folder at ~/.segnlp/<id> and create new one
                )


Processing and Storing Dataset: 100%|██████████| 402/402 [01:07<00:00,  5.96it/s]


In [3]:
# Hyperparamaters
# one set of hyperparamaters per layer + general hyperaparamaters
hps = {
        "general":{
                "optimizer": "Adam",
                "lr": 0.001,
                "batch_size": 32,
                "max_epochs":100,
                "patience": 10,
                "task_weight": 0.5,
                "use_target_segs_k": 10, # sampling
                #"freeze_segment_module_k": 5, # Havent tested this fully yet. (Next on the list)
                },

       "LSTM": {   
                    "input_dropout": 0.5,
                    "dropout":0.5,
                    "hidden_size": 100,
                    "num_layers":1,
                    "bidir":True,
                    },
        "BigramSeg": {
                        "hidden_size": 512,
                },
        "Agg":{
                "mode":"mean",
                },
        "DepTreeLSTM": {
                        "dropout":0.5,
                        "hidden_size":100,
                        "bidir":True,
                        "mode": "shortest_path",
                        },
        "LinearPairEnc": {
                        "hidden_size":100,
                        "activation": "Tanh",
                        },
        "DirLinkLabeler": {
                            "dropout": 0.5,
                            "match_threshold": 0.5,
                        }
        }


In [ ]:
# Will train models for each random seed and each set of hyperparamaters given. All models are saved
# All outputs are also logged to ~/.segnlp/<id>/logs/<n>.log but havent fully test this either.
best_hp = exp.train(
                        hyperparamaters = hps,
                        n_random_seeds=6,
                        monitor_metric="val_f1-micro"
                        )

In [ ]:
#TODO


# 1 ) Make sure logging of all scores is working
# 2 ) finish the code for exp.test()
# 3 ) tie the statistical_significance testing fucntions to logs, so one can get the best hyperparamaters
#     quicker and also test one model against another
# 4)  add some code for plotting logs